# 🔬 Análise Interativa de Energia Sísmica - Por Canais

Esta aplicação permite analisar a **energia total** dos dados sísmicos **por canais (G, H, E)** de forma interativa. A energia é calculada usando a fórmula **E = √(x² + y² + z²)** e apresentada em histogramas de distribuição de frequência.

## 📋 Como usar:
1. **Carregue seu arquivo**: Use o botão abaixo para selecionar um arquivo CSV
2. **Aguarde o processamento**: A análise de energia será executada automaticamente por canais
3. **Visualize os resultados**: Histogramas de energia total por canal com frequências
4. **Baixe os resultados**: Arquivo processado estará disponível para download

## ⚡ Cálculo de Energia:
- **Fórmula**: E = √(x_amplitude² + y_amplitude² + z_amplitude²)
- **Análise**: Distribuição de energia total por canal
- **Histogramas**: Energia no eixo X, Frequência no eixo Y

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import io
import base64
from datetime import datetime
import warnings

# Configurações
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas carregadas com sucesso!")

In [ ]:
# Widget para upload de arquivo
file_upload = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Carregar CSV',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Botão para processar
process_button = widgets.Button(
    description='🔄 Processar Análise',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='40px')
)

# Área de status
status_output = widgets.Output()

# Área de resultados
results_output = widgets.Output()

# Interface
display(widgets.VBox([
    widgets.HTML("<h3>📁 Carregar Arquivo de Dados Sísmicos</h3>"),
    widgets.HTML("<p>Selecione um arquivo CSV ortogonalizado com dados de eventos sísmicos para análise de energia total por canal.</p>"),
    widgets.HTML("<p><strong>Formato esperado:</strong> event_id, plane, sensor, x_amplitude, y_amplitude, z_amplitude</p>"),
    widgets.HTML("<p><strong>Análise:</strong> Energia  por canal com histogramas de frequência</p>"),
    file_upload,
    process_button,
    status_output,
    results_output
]))

In [ ]:
def validate_csv_format(df):
    """Valida se o CSV está no formato correto"""
    required_columns = ['event_id', 'plane', 'sensor', 'x_amplitude', 'y_amplitude', 'z_amplitude']
    
    # Verificar colunas obrigatórias
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        return False, f"Colunas faltando: {missing_columns}"
    
    # Verificar se amplitudes são numéricas
    amplitude_cols = ['x_amplitude', 'y_amplitude', 'z_amplitude']
    for col in amplitude_cols:
        if not pd.api.types.is_numeric_dtype(df[col]):
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                return False, f"Coluna {col} não pode ser convertida para numérico"
    
    # Verificar valores nulos
    null_counts = df[amplitude_cols].isnull().sum()
    if null_counts.sum() > 0:
        return False, f"Valores nulos encontrados: {null_counts.to_dict()}"
    
    # Verificar se há planos
    if df['plane'].isnull().all():
        return False, "Coluna 'plane' não contém dados válidos"
    
    return True, "Formato válido"

def calculate_energy_analysis(df):
    """Calcula análise de energia total por canal"""
    
    # Debug: verificar dados de entrada
    print(f"🔍 Debug - Colunas: {df.columns.tolist()}")
    print(f"🔍 Debug - Canais únicos: {df['plane'].unique()}")
    print(f"🔍 Debug - Total de registros: {len(df)}")
    
    # Calcular energia total usando fórmula E = √(x² + y² + z²)
    df['total_energy'] = np.sqrt(
        df['x_amplitude']**2 + 
        df['y_amplitude']**2 + 
        df['z_amplitude']**2
    )
    
    print(f"🔍 Debug - Energia calculada com sucesso!")
    print(f"🔍 Debug - Energia mínima: {df['total_energy'].min():.6e}")
    print(f"🔍 Debug - Energia máxima: {df['total_energy'].max():.6e}")
    print(f"🔍 Debug - Energia média: {df['total_energy'].mean():.6e}")
    
    # Estatísticas por canal/plano
    channel_stats = {}
    channels = sorted(df['plane'].unique())
    
    for channel in channels:
        channel_data = df[df['plane'] == channel]['total_energy']
        channel_stats[channel] = {
            'count': len(channel_data),
            'mean': channel_data.mean(),
            'std': channel_data.std(),
            'min': channel_data.min(),
            'max': channel_data.max(),
            'median': channel_data.median(),
            'q25': channel_data.quantile(0.25),
            'q75': channel_data.quantile(0.75),
            'range': channel_data.max() - channel_data.min()
        }
        
        print(f"🔍 Debug - Canal {channel}: {len(channel_data)} registros")
        print(f"    Energia média: {channel_stats[channel]['mean']:.6e}")
        print(f"    Desvio padrão: {channel_stats[channel]['std']:.6e}")
    
    # Adicionar estatísticas ao DataFrame
    for channel in channels:
        channel_mask = df['plane'] == channel
        stats = channel_stats[channel]
        
        df.loc[channel_mask, 'channel_energy_mean'] = stats['mean']
        df.loc[channel_mask, 'channel_energy_std'] = stats['std']
        df.loc[channel_mask, 'channel_energy_median'] = stats['median']
    
    # Adicionar ranking de energia
    df['energy_rank'] = df['total_energy'].rank(ascending=False)
    df['energy_percentile'] = df['total_energy'].rank(pct=True)
    
    # Classificar energia por intensidade (por canal)
    for channel in channels:
        channel_mask = df['plane'] == channel
        channel_data = df[channel_mask]['total_energy']
        
        if len(channel_data) > 0 and channel_data.std() > 0:
            low_threshold = channel_data.quantile(0.33)
            high_threshold = channel_data.quantile(0.67)
            
            def classify_energy(value):
                if value <= low_threshold:
                    return 'Baixa'
                elif value <= high_threshold:
                    return 'Média'
                else:
                    return 'Alta'
            
            df.loc[channel_mask, 'energy_intensity'] = channel_data.apply(classify_energy)
        else:
            df.loc[channel_mask, 'energy_intensity'] = 'Média'
    
    # Identificar alta energia (percentil 95)
    high_energy_threshold = df['total_energy'].quantile(0.95)
    df['high_energy_flag'] = df['total_energy'] > high_energy_threshold
    
    # Debug final
    print(f"🔍 Debug - Registros com alta energia: {df['high_energy_flag'].sum()}")
    print(f"🔍 Debug - Distribuição de intensidade: {df['energy_intensity'].value_counts().to_dict()}")
    
    return df, channel_stats

def create_energy_plots(df, channel_stats):
    """Cria gráficos de análise de energia total por canal"""
    
    # Configurar figura principal
    fig = plt.figure(figsize=(18, 15))
    
    channels = sorted(df['plane'].unique())
    
    # Cores para cada canal
    channel_colors = {'G': '#2E8B57', 'H': '#FF6347', 'E': '#4169E1', 
                     'A': '#9932CC', 'B': '#FF8C00', 'C': '#DC143C'}
    
    # Calcular bins globais para manter escala consistente
    all_energies = df['total_energy'].values
    min_energy = all_energies.min()
    max_energy = all_energies.max()
    
    # Usar escala logarítmica se há grande variação
    if max_energy / min_energy > 1000:
        bins = np.logspace(np.log10(min_energy + 1e-12), np.log10(max_energy), 50)
        use_log_scale = True
    else:
        bins = np.linspace(min_energy, max_energy, 50)
        use_log_scale = False
    
    # 1-3. Histogramas individuais de energia por canal
    n_channels = len(channels)
    n_cols = min(3, n_channels)
    n_rows = (n_channels + n_cols - 1) // n_cols
    
    for i, channel in enumerate(channels):
        plt.subplot(3, 3, i+1)
        
        # Filtrar dados do canal atual
        channel_data = df[df['plane'] == channel]['total_energy']
        
        if len(channel_data) > 0:
            # Criar histograma
            counts, bin_edges, patches = plt.hist(channel_data, bins=bins, 
                                                 color=channel_colors.get(channel, '#808080'), 
                                                 alpha=0.7, 
                                                 edgecolor='black', 
                                                 linewidth=0.5)
            
            # Configurar eixos
            plt.xlabel('Energia Total E = √(x² + y² + z²)')
            plt.ylabel('Frequência')
            plt.title(f'Canal {channel}\n({len(channel_data):,} registros)')
            
            # Usar escala logarítmica se necessário
            if use_log_scale:
                plt.xscale('log')
            
            # Adicionar grid
            plt.grid(True, alpha=0.3)
            
            # Estatísticas do canal
            mean_energy = channel_data.mean()
            median_energy = channel_data.median()
            
            # Linha vertical para média
            plt.axvline(mean_energy, color='red', linestyle='--', 
                       linewidth=2, label=f'Média: {mean_energy:.2e}')
            
            # Linha vertical para mediana
            plt.axvline(median_energy, color='orange', linestyle=':', 
                       linewidth=2, label=f'Mediana: {median_energy:.2e}')
            
            plt.legend(fontsize=8)
    
    # 4. Histograma sobreposto comparativo
    plt.subplot(3, 3, 4)
    
    for channel in channels:
        channel_data = df[df['plane'] == channel]['total_energy']
        if len(channel_data) > 0:
            plt.hist(channel_data, bins=bins, alpha=0.6, 
                    label=f'Canal {channel} (n={len(channel_data):,})', 
                    color=channel_colors.get(channel, '#808080'), 
                    edgecolor='black')
    
    plt.xlabel('Energia Total E = √(x² + y² + z²)')
    plt.ylabel('Frequência')
    plt.title('Comparação de Distribuições entre Canais')
    plt.legend()
    plt.grid(True, alpha=0.3)
    if use_log_scale:
        plt.xscale('log')
    plt.yscale('log')
    
    # 5. Comparação de estatísticas entre canais
    plt.subplot(3, 3, 5)
    x_pos = np.arange(len(channels))
    bar_width = 0.35
    
    means = [channel_stats[ch]['mean'] for ch in channels]
    stds = [channel_stats[ch]['std'] for ch in channels]
    
    plt.bar(x_pos - bar_width/2, means, bar_width, 
            label='Média', alpha=0.7, color='skyblue')
    plt.bar(x_pos + bar_width/2, stds, bar_width, 
            label='Desvio Padrão', alpha=0.7, color='lightcoral')
    
    plt.xlabel('Canais')
    plt.ylabel('Energia')
    plt.title('Comparação de Médias e Desvios por Canal')
    plt.xticks(x_pos, channels)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    # 6. Box plot comparativo
    plt.subplot(3, 3, 6)
    
    energy_data = []
    for channel in channels:
        channel_data = df[df['plane'] == channel]['total_energy']
        if len(channel_data) > 0:
            energy_data.append(channel_data.values)
    
    if energy_data:
        box_plot = plt.boxplot(energy_data, labels=channels, patch_artist=True)
        
        # Colorir por canal
        for i, patch in enumerate(box_plot['boxes']):
            if i < len(channels):
                patch.set_facecolor(channel_colors.get(channels[i], '#808080'))
                patch.set_alpha(0.7)
    
    plt.ylabel('Energia Total')
    plt.title('Distribuição Estatística por Canal')
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    
    # 7. Intervalos de energia mais populares
    plt.subplot(3, 3, 7)
    
    # Definir intervalos baseados na distribuição global
    n_intervals = 10
    intervals = np.linspace(min_energy, max_energy, n_intervals + 1)
    
    # Contar frequências por intervalo e por canal
    interval_data = []
    for i in range(len(intervals)-1):
        start = intervals[i]
        end = intervals[i+1]
        
        counts = []
        for channel in channels:
            channel_data = df[df['plane'] == channel]['total_energy']
            count = len(channel_data[(channel_data >= start) & (channel_data < end)])
            counts.append(count)
        
        interval_data.append(counts)
    
    interval_data = np.array(interval_data).T
    
    # Criar gráfico de barras empilhadas
    x_pos = np.arange(len(intervals)-1)
    bottom = np.zeros(len(intervals)-1)
    
    for i, channel in enumerate(channels):
        plt.bar(x_pos, interval_data[i], bottom=bottom, 
                label=f'Canal {channel}', 
                color=channel_colors.get(channel, '#808080'), 
                alpha=0.7)
        bottom += interval_data[i]
    
    plt.xlabel('Intervalos de Energia')
    plt.ylabel('Frequência')
    plt.title('Distribuição por Intervalos de Energia')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 8. Heatmap de estatísticas por canal
    plt.subplot(3, 3, 8)
    
    # Criar matriz de estatísticas para heatmap
    stats_names = ['mean', 'std', 'min', 'max']
    heatmap_data = []
    
    for channel in channels:
        row = []
        for stat in stats_names:
            row.append(channel_stats[channel][stat])
        heatmap_data.append(row)
    
    if heatmap_data:
        heatmap_data = np.array(heatmap_data)
        im = plt.imshow(heatmap_data, cmap='viridis', aspect='auto')
        plt.colorbar(im, shrink=0.8)
        plt.xticks(range(len(stats_names)), 
                  [stat.title() for stat in stats_names])
        plt.yticks(range(len(channels)), channels)
        plt.title('Heatmap de Estatísticas por Canal')
        
        # Adicionar valores no heatmap
        for i in range(len(channels)):
            for j in range(len(stats_names)):
                plt.text(j, i, f'{heatmap_data[i, j]:.2e}', 
                        ha='center', va='center', color='white', fontweight='bold', fontsize=8)
    
    # 9. Distribuição de registros e energia por canal
    plt.subplot(3, 3, 9)
    
    channel_counts = df['plane'].value_counts()
    colors = [channel_colors.get(channel, '#808080') for channel in channel_counts.index]
    
    plt.pie(channel_counts.values, labels=channel_counts.index, 
            autopct='%1.1f%%', colors=colors, startangle=90)
    plt.title('Distribuição de Registros por Canal')
    
    plt.tight_layout()
    plt.show()
    
    return fig

print("✅ Funções de análise de energia por canal carregadas!")

In [ ]:
def generate_energy_summary_stats(df, channel_stats):
    """Gera estatísticas resumidas de energia por canal"""
    
    total_records = len(df)
    unique_events = df['event_id'].nunique()
    unique_sensors = df['sensor'].nunique()
    unique_channels = df['plane'].nunique()
    
    channels = sorted(df['plane'].unique())
    
    # Estatísticas globais de energia
    global_energy_mean = df['total_energy'].mean()
    global_energy_std = df['total_energy'].std()
    global_energy_min = df['total_energy'].min()
    global_energy_max = df['total_energy'].max()
    high_energy_count = df['high_energy_flag'].sum()
    
    stats_html = f"""
    <div style="background-color: #f0f8ff; padding: 20px; border-radius: 10px; margin: 10px 0;">
        <h2>⚡ Resumo da Análise de Energia Total por Canais</h2>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px;">
            <div>
                <h3>📈 Estatísticas Gerais</h3>
                <ul>
                    <li><strong>Total de registros:</strong> {total_records:,}</li>
                    <li><strong>Eventos únicos:</strong> {unique_events:,}</li>
                    <li><strong>Sensores únicos:</strong> {unique_sensors}</li>
                    <li><strong>Canais únicos:</strong> {unique_channels} ({', '.join(channels)})</li>
                </ul>
                
                <h3>⚡ Energia Global</h3>
                <ul>
                    <li><strong>Energia média:</strong> {global_energy_mean:.2e}</li>
                    <li><strong>Desvio padrão:</strong> {global_energy_std:.2e}</li>
                    <li><strong>Energia mínima:</strong> {global_energy_min:.2e}</li>
                    <li><strong>Energia máxima:</strong> {global_energy_max:.2e}</li>
                    <li><strong>Alta energia (P95):</strong> {high_energy_count} registros</li>
                </ul>
            </div>
            <div>
                <h3>📋 Distribuição por Canal</h3>
                <ul>
    """
    
    # Adicionar distribuição por canal
    channel_counts = df['plane'].value_counts()
    for channel in channels:
        count = channel_counts.get(channel, 0)
        percentage = count / total_records * 100
        mean_energy = channel_stats[channel]['mean']
        stats_html += f"<li><strong>Canal {channel}:</strong> {count:,} ({percentage:.1f}%) - Energia: {mean_energy:.2e}</li>"
    
    stats_html += """
                </ul>
            </div>
        </div>
        <div style="margin-top: 20px;">
            <h3>🔬 Estatísticas Detalhadas de Energia por Canal</h3>
            <div style="overflow-x: auto;">
                <table style="width: 100%; border-collapse: collapse; margin-top: 10px;">
                    <tr style="background-color: #e6f3ff;">
                        <th style="border: 1px solid #ddd; padding: 8px;">Canal</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Registros</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Energia Média</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Desvio Padrão</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Mínimo</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Máximo</th>
                        <th style="border: 1px solid #ddd; padding: 8px;">Range</th>
                    </tr>
    """
    
    # Adicionar estatísticas por canal
    for i, channel in enumerate(channels):
        stats = channel_stats[channel]
        row_style = "background-color: #f9f9f9;" if i % 2 == 0 else ""
        
        stats_html += f"""
                <tr style="{row_style}">
                    <td style="border: 1px solid #ddd; padding: 8px;"><strong>{channel}</strong></td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{stats['count']:,}</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{stats['mean']:.2e}</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{stats['std']:.2e}</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{stats['min']:.2e}</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{stats['max']:.2e}</td>
                    <td style="border: 1px solid #ddd; padding: 8px;">{stats['range']:.2e}</td>
                </tr>
        """
    
    stats_html += """
                </table>
            </div>
        </div>
        
        <div style="margin-top: 20px; background-color: #fff3cd; padding: 15px; border-radius: 8px;">
            <h3>📊 Interpretação dos Resultados</h3>
            <ul>
                <li><strong>Energia Total:</strong> Calculada usando E = √(x² + y² + z²)</li>
                <li><strong>Histogramas:</strong> Mostram a distribuição de frequência de energia por canal</li>
                <li><strong>Eixo X:</strong> Valores de energia total</li>
                <li><strong>Eixo Y:</strong> Frequência de ocorrência</li>
                <li><strong>Comparação:</strong> Permite identificar diferenças entre canais G, H, E</li>
            </ul>
        </div>
    </div>
    """
    
    return stats_html

def create_download_link(df, filename):
    """Cria link para download do CSV processado"""
    csv_string = df.to_csv(index=False)
    b64 = base64.b64encode(csv_string.encode()).decode()
    
    # Contar novas colunas adicionadas
    original_cols = ['event_id', 'plane', 'sensor', 'x_amplitude', 'y_amplitude', 'z_amplitude']
    new_cols = [col for col in df.columns if col not in original_cols]
    
    download_html = f"""
    <div style="background-color: #e8f5e8; padding: 15px; border-radius: 8px; margin: 10px 0;">
        <h3>💾 Download dos Resultados da Análise de Energia</h3>
        <p>Clique no link abaixo para baixar o arquivo CSV com todas as análises de energia por canal:</p>
        <a href="data:text/csv;base64,{b64}" download="{filename}" 
           style="background-color: #4CAF50; color: white; padding: 10px 20px; 
                  text-decoration: none; border-radius: 5px; font-weight: bold;">
           📥 Baixar {filename}
        </a>
        <p style="margin-top: 10px; font-size: 0.9em; color: #666;">
            Arquivo contém {len(df)} registros com análise de energia total por canal.
        </p>
        <p style="font-size: 0.9em; color: #666;">
            <strong>Novas colunas adicionadas:</strong> {len(new_cols)} colunas incluindo total_energy, 
            estatísticas por canal, rankings e classificações.
        </p>
        
        <div style="margin-top: 10px; font-size: 0.8em; color: #555;">
            <strong>Colunas principais:</strong><br>
            • <code>total_energy</code> - Energia total calculada<br>
            • <code>channel_energy_mean</code> - Média de energia do canal<br>
            • <code>energy_intensity</code> - Classificação de intensidade<br>
            • <code>high_energy_flag</code> - Flag para alta energia<br>
            • <code>energy_rank</code> - Ranking de energia<br>
            • <code>energy_percentile</code> - Percentil de energia
        </div>
    </div>
    """
    
    return download_html

def process_analysis(button):
    """Função principal de processamento para análise de energia"""
    
    with status_output:
        clear_output()
        
        if not file_upload.value:
            print("❌ Por favor, selecione um arquivo CSV primeiro.")
            return
        
        try:
            print("🔄 Carregando arquivo...")
            
            # Carregar arquivo
            uploaded_file = list(file_upload.value.values())[0]
            content = uploaded_file['content']
            filename = uploaded_file['metadata']['name']
            
            # Ler CSV
            df = pd.read_csv(io.BytesIO(content))
            print(f"✅ Arquivo carregado: {filename} ({len(df)} registros)")
            
            # Validar formato
            print("🔍 Validando formato do arquivo...")
            is_valid, validation_message = validate_csv_format(df)
            
            if not is_valid:
                print(f"❌ Erro de formato: {validation_message}")
                print("\n📋 Formato esperado:")
                print("   - event_id (numérico)")
                print("   - plane (texto: G, H, E, etc.)")
                print("   - sensor (texto)")
                print("   - x_amplitude (numérico)")
                print("   - y_amplitude (numérico)")
                print("   - z_amplitude (numérico)")
                return
            
            print("✅ Formato válido!")
            
            # Processar análise de energia
            print("⚡ Calculando energia total por canal...")
            print("   Fórmula: E = √(x² + y² + z²)")
            df_processed, channel_stats = calculate_energy_analysis(df)
            print("✅ Análise de energia concluída!")
            
            # Gerar timestamp para nome do arquivo
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"analise_energia_canais_{timestamp}.csv"
            
            print(f"📊 Gerando visualizações e relatório...")
            
        except Exception as e:
            print(f"❌ Erro durante o processamento: {str(e)}")
            import traceback
            print(f"Detalhes do erro: {traceback.format_exc()}")
            return
    
    # Mostrar resultados
    with results_output:
        clear_output()
        
        try:
            # Estatísticas resumidas
            stats_html = generate_energy_summary_stats(df_processed, channel_stats)
            display(HTML(stats_html))
            
            # Gráficos de energia
            print("\n📈 Histogramas de Energia Total por Canal:")
            print("   • Eixo X: Energia Total E = √(x² + y² + z²)")
            print("   • Eixo Y: Frequência de ocorrência")
            create_energy_plots(df_processed, channel_stats)
            
            # Link de download
            download_html = create_download_link(df_processed, output_filename)
            display(HTML(download_html))
            
            # Mostrar primeiras linhas dos resultados
            print("\n📋 Primeiras linhas dos resultados processados:")
            
            # Selecionar colunas mais relevantes para exibir
            display_cols = ['event_id', 'plane', 'sensor', 'x_amplitude', 'y_amplitude', 'z_amplitude', 'total_energy']
            
            # Adicionar colunas de estatísticas se existirem
            energy_cols = [col for col in df_processed.columns if 'energy' in col and col not in display_cols]
            display_cols.extend(energy_cols[:4])  # Limitar para não sobrecarregar a exibição
            
            # Filtrar apenas colunas que existem
            available_cols = [col for col in display_cols if col in df_processed.columns]
            
            display(df_processed[available_cols].head(10))
            
            # Análise estatística rápida
            print(f"\n🔬 Análise Rápida:")
            print(f"   • Energia total média: {df_processed['total_energy'].mean():.2e}")
            print(f"   • Energia total máxima: {df_processed['total_energy'].max():.2e}")
            print(f"   • Coeficiente de variação: {(df_processed['total_energy'].std() / df_processed['total_energy'].mean()):.3f}")
            
            # Comparação entre canais
            print(f"\n📊 Comparação entre Canais:")
            for channel in sorted(df_processed['plane'].unique()):
                channel_data = df_processed[df_processed['plane'] == channel]['total_energy']
                print(f"   • Canal {channel}: {len(channel_data)} registros, energia média = {channel_data.mean():.2e}")
            
        except Exception as e:
            print(f"❌ Erro ao gerar visualizações: {str(e)}")
            import traceback
            print(f"Detalhes do erro: {traceback.format_exc()}")
    
    with status_output:
        print("🎉 Análise de energia por canais completa! Verifique os resultados abaixo.")

# Conectar função ao botão
process_button.on_click(process_analysis)

print("✅ Interface interativa para análise de energia por canais pronta!")

---

## 📖 Sobre a Análise de Energia Total por Canais

### ⚡ **Cálculo de Energia:**
- **Fórmula Principal**: E = √(x_amplitude² + y_amplitude² + z_amplitude²)
- **Análise por Canal**: Separação e análise dos dados por canal (G, H, E)
- **Histogramas de Frequência**: Distribuição de energia total no eixo X, frequência no eixo Y
- **Estatísticas Comparativas**: Média, desvio padrão, mínimo, máximo por canal

### 📊 **Tipos de Gráficos Gerados:**
- **Histogramas Individuais**: Um histograma para cada canal mostrando distribuição de energia
- **Histograma Comparativo**: Sobreposição de todos os canais para comparação direta  
- **Gráficos de Barras**: Comparação de médias e desvios padrão entre canais
- **Box Plots**: Distribuições estatísticas comparativas por canal
- **Heatmaps**: Visualização de estatísticas resumidas por canal
- **Gráfico de Pizza**: Distribuição de registros por canal

### ? **Critérios de Análise:**
- **Energia Total**: Magnitude resultante dos três componentes de amplitude
- **Distribuição por Canal**: Análise separada para cada canal/plano geométrico
- **Intervalos de Energia**: Histogramas mostrando frequência por faixa de valores de energia
- **Classificação de Intensidade**: Baixa, Média, Alta energia por canal
- **Detecção de Alta Energia**: Registros acima do percentil 95

### 💾 **Dados de Saída:**
O arquivo CSV gerado contém todas as colunas originais mais:
- `total_energy` - Energia total calculada E = √(x² + y² + z²)
- `channel_energy_mean` - Média de energia do canal
- `channel_energy_std` - Desvio padrão de energia do canal  
- `channel_energy_median` - Mediana de energia do canal
- `energy_rank` - Ranking de energia (1 = maior energia)
- `energy_percentile` - Percentil de energia (0-1)
- `energy_intensity` - Classificação de intensidade (Baixa/Média/Alta)
- `high_energy_flag` - Flag para registros de alta energia (percentil 95+)

### 📈 **Interpretação dos Histogramas:**
- **Eixo Horizontal (X)**: Valores de energia total E = √(x² + y² + z²)
- **Eixo Vertical (Y)**: Frequência de ocorrência (quantas vezes cada faixa de energia aparece)
- **Comparação entre Canais**: Permite identificar diferenças na distribuição de energia entre canais G, H, E
- **Picos**: Indicam faixas de energia mais comuns em cada canal
- **Caudas**: Mostram eventos de energia extrema (muito baixa ou muito alta)

### 🎯 **Aplicações Práticas:**
- **Caracterização de Canais**: Identificar qual canal tem maior/menor energia média
- **Detecção de Anomalias**: Eventos com energia muito alta ou baixa
- **Comparação Estatística**: Diferenças significativas entre canais
- **Classificação Automática**: Separação de eventos por intensidade energética
- **Controle de Qualidade**: Identificação de possíveis problemas nos sensores

---
*Desenvolvido para análise de energia sísmica por canais - Projeto Braskem*